In [1]:
import os
import sys
import comtypes.client
import numpy as np
import math
import pandas as pd
import random as r
from collections import Counter
from scipy.special import gamma
from openpyxl import load_workbook

import lightgbm as lgb
import xgboost as xgb
from scipy.interpolate import Rbf
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import copy
import time

## Input

In [2]:
class Config:
    pass

config=Config()
config.concrete_cover=40
# beam parameter
config.long_beam_dia=16
config.conf_beam_dia=13
config.beam_conf_leg=2

# column parameter
config.long_col_dia=25
config.conf_col_dia=16
config.col_conf_leg=2

allowable_drift = 1000#0.02 no limit

safe_proba_threshold = (0.1, 0.2)
threshold_multiplier = (2, 1.5)

In [3]:
# constants
steel_density = 5400 #kg/m3 7850-2450
g = 9.80665 # g constant

### Misc. Functions

In [4]:
def calc_area(dia):
    return math.pi/4*dia**2

def floor_to_multiple(number, multiple):
    return multiple*math.floor(number/multiple)

def ceil_to_multiple(number, multiple):
    return multiple*math.ceil(number/multiple)

def round_to_multiple(number, multiple):
    return multiple*round(number/multiple)

def numpy_round_to_multiple(array, multiple):
    return multiple*np.round(array/multiple)

# Classes
### ETABS Class

In [5]:
class ETABSObjectiveFunction:
    def __init__(self):
        self.combo_names = None
        self.frame_names = None
        self.frame_prop = None # conjunction with 
        self.joint_names = None
        self.prop_names = None
        self.story_names = None
        self.fstory_dict =None
        self.EQ_coeff = None # R, Cd, I
        
        self.fprop_dict = None
        self.beam_props = None
        self.column_props = None
        self.fprop_size = None # depth, width
        
        self.drift_failure = 0
        self.max_story_drift = 0
        
        self.beam_design_groups = []
        self.column_design_groups = []
        
        self.rebar_weight = 0
        self.design_failure = 0
        self.fail_dict = Counter([])
        self.fail_member = []
        self.concrete_weight = 0
        
        #set the following flag to True to attach to an existing instance of the program
        #otherwise a new instance of the program will be started
        AttachToInstance = False
        ######### CHANGE HERE
        fullModelPath = "C:/Users/E2-415/Desktop/ETABS Python/Model/Model1story7v3.edb"
        
        #create API helper object
        helper = comtypes.client.CreateObject('ETABSv1.Helper')
        helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
        
        if AttachToInstance:
            #attach to a running instance of ETABS
            try:
                #get the active ETABS object
                self.myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
            except (OSError, comtypes.COMError):
                print("No running instance of the program found or failed to attach.")
                sys.exit(-1)
        else:
            try: 
                #create an instance of the ETABS object from the latest installed ETABS
                self.myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
            except (OSError, comtypes.COMError):
                print("Cannot start a new instance of the program.")
                sys.exit(-1)
            #start ETABS application
            self.myETABSObject.ApplicationStart()

        
        #create SapModel object
        self.sapModel = self.myETABSObject.SapModel

        # 2. Run SAP Model
        self.sapModel.File.OpenFile(fullModelPath)
        # set units
        kn_mm_C = 5
        if self.sapModel.GetPresentUnits() != kn_mm_C:
            self.sapModel.SetPresentUnits(kn_mm_C)

        ## 3. Get Information 
        _ = self.get_information()
        
        # save run
#         self.sapModel.File.Save()
#         self.sapModel.Analyze.RunAnalysis()
               
    def generate_frame(self):
        ######### CHANGE HERE
        # generate default beam and column frame prop for each story (exterior and interior)
        cover_beam = config.concrete_cover+config.conf_beam_dia+config.long_beam_dia/2
        cover_col = config.concrete_cover
        PropFrame = self.sapModel.PropFrame
        long_size=str(config.long_col_dia)
        conf_size=str(config.conf_col_dia)
        for e, _ in enumerate(etabsClass.story_names):
            for e, _ in enumerate(etabsClass.story_names):
                name="B"+str(e+1)+"ExtX"
                PropFrame.SetRectangle(name, "Concrete", 600, 400) # mm
                PropFrame.SetRebarBeam(name, "Rebar", "Rebar", cover_beam, cover_beam, 0, 0, 0, 0)

                name="B"+str(e+1)+"ExtY"
                PropFrame.SetRectangle(name, "Concrete", 600, 400) # mm
                PropFrame.SetRebarBeam(name, "Rebar", "Rebar", cover_beam, cover_beam, 0, 0, 0, 0)
                
                name="B"+str(e+1)+"IntX"
                PropFrame.SetRectangle(name, "Concrete", 600, 400) # mm
                PropFrame.SetRebarBeam(name, "Rebar", "Rebar", cover_beam, cover_beam, 0, 0, 0, 0)
                
                name="B"+str(e+1)+"IntY"
                PropFrame.SetRectangle(name, "Concrete", 600, 400) # mm
                PropFrame.SetRebarBeam(name, "Rebar", "Rebar", cover_beam, cover_beam, 0, 0, 0, 0)

    
    def modify_frame(self):
        ######### CHANGE HERE
        # modify frame section based on self.fprop_size
        etabsClass.sapModel.SetModelIsLocked(False)
        cover_beam = config.concrete_cover+config.conf_beam_dia+config.long_beam_dia/2
        cover_col = config.concrete_cover
        long_size=str(config.long_col_dia)
        conf_size=str(config.conf_col_dia)
        for prop_name in list(self.fprop_size.keys()):
            self.sapModel.PropFrame.SetRectangle(prop_name, "Concrete", self.fprop_size[prop_name][0], 
                                                 self.fprop_size[prop_name][1])
            if prop_name[0]=='B':
                self.sapModel.PropFrame.SetRebarBeam(prop_name, "Rebar", "Rebar", cover_beam, cover_beam, 0, 0, 0, 0)
            else:
                self.sapModel.PropFrame.SetRebarColumn(prop_name, "Rebar", "Rebar", 1, 1, cover_col, 0, 4, 4, 
                                         long_size, conf_size, 150, 3, 3, True)
        
#         for name in self.frame_names:
#                 self.sapModel.DesignConcrete.ACI318_14.SetOverwrite(name, 1, 2) # intermediate sway
#                 etabsClass.sapModel.DesignConcrete.ACI318_08_IBC2009.SetOverwrite(name, 1, 2) # intermediate sway
    
    def get_information(self):
        # get all information from etabs model
        frames_data = self.sapModel.FrameObj.GetAllFrames()
        self.frame_names = frames_data[1]
        self.frame_prop = frames_data[2]
        self.fstory_dict = dict(zip(frames_data[1],frames_data[3]))
        
        joints_data = self.sapModel.PointObj.GetAllPoints()
        self.joint_names = joints_data[1]
        
        _, self.combo_names, _= self.sapModel.RespCombo.GetNameList()
        
        props_data = self.sapModel.PropFrame.GetAllFrameProperties()
        self.prop_names = props_data[1]
        
        stories_data = self.sapModel.Story.GetNameList()
        self.story_names = stories_data[1]
        
        fprop_data=self.sapModel.PropFrame.GetAllFrameProperties()
        fprop_names = fprop_data[1]
        frame_dict={}
        frame_size_dict={}
        for i, name in enumerate(fprop_names):
            frame_dict[name]=[]
            frame_size_dict[name]=(fprop_data[3][i],fprop_data[4][i])
        self.fprop_size = frame_size_dict
        
        for idx, name in enumerate(self.frame_prop):
            frame_dict[name].append(self.frame_names[idx])
        self.frame_dict=frame_dict
        self.beam_props=[name for name in list(self.frame_dict.keys()) if name[0] == 'B']
        self.column_props=[name for name in list(self.frame_dict.keys()) if name[0] == 'C']
        
        ######### CHANGE HERE
        self.EQ_coeff = [5, 4.5, 1]#[R, Cd, I]
        return None
    
    def get_weight(self, design_sizes):
        for idx, prop_name in enumerate(self.fprop_size.keys()):
            self.fprop_size[prop_name]=(design_sizes[2*idx],design_sizes[2*idx+1])
        self.modify_frame()
        
        objs=self.sapModel.DatabaseTables.GetTableForDisplayArray('Material List by Object Type', [], 'All')
        self.concrete_weight=(float(objs[4][2])+float(objs[4][5+2])+float(objs[4][10+2]))*1000/g # kg  floor
        return self.concrete_weight
    
    def check_drift(self, design_drift = None):
        if design_drift == None:
            design_drift=allowable_drift
            
        # check elastic drift according to building earthquake category
        if not self.sapModel.GetModelIsLocked():
            self.sapModel.Analyze.RunAnalysis()
        sapResult = self.sapModel.Results # result need to be available (run)
        sapResult.Setup.DeselectAllCasesAndCombosForOutput()
        sapResult.Setup.SetCaseSelectedForOutput("EQ")
        
        # reset
        self.drift_failure=0
        self.max_story_drift=0
        
        drift_data = sapResult.StoryDrifts()
        for story_name in self.story_names:
            drifts = [drift_data[6][i] for i, e in enumerate(list(drift_data[1])) if e == story_name]
            if not drifts: # drift is empty
                drifts=[0]
            Cd=self.EQ_coeff[1]
            I=self.EQ_coeff[2]
            max_drift=max(drifts)*Cd/I
            if max_drift > self.max_story_drift:
                self.max_story_drift = max_drift
            if max_drift > design_drift:
                self.drift_failure += 1
        return None
    
    def check_design(self, design_config=None):
        if design_config == None:
            design_config = config
            
        if not self.sapModel.DesignConcrete.GetResultsAvailable():
            if not self.sapModel.GetModelIsLocked():
                self.sapModel.Analyze.RunAnalysis()
            self.sapModel.DesignConcrete.StartDesign()
        
        # reset
        self.beam_design_groups=[]
        self.column_design_groups=[]
        self.rebar_weight=0
        self.design_failure=0
        self.fail_dict = Counter([])
        self.fail_member=[]
        
        # Concrete
        objs=self.sapModel.DatabaseTables.GetTableForDisplayArray('Material List by Object Type', [], 'All')
        self.concrete_weight=(float(objs[4][2])+float(objs[4][5+2])+float(objs[4][10+2]))*1000/g # kg 
        
        # Rebar - beam design
        for prop_name in self.beam_props:
            size=self.fprop_size[prop_name]
            design_group=DesignGroup()
            for frame_name in self.frame_dict[prop_name]:
                objs=self.sapModel.DesignConcrete.GetSummaryResultsBeam(frame_name)
                beam_design = BeamDesign(objs, design_config, size, prop_name)
                design_group.append(beam_design)
            design_group.calculate_rebar_weight()
            self.beam_design_groups.append(design_group)
        self.rebar_weight += sum([design_group.rebar_weight for design_group in self.beam_design_groups])
        self.design_failure += sum([design_group.fail_num for design_group in self.beam_design_groups])
        
        for design_group in self.beam_design_groups:
            if len(design_group.fail_dict.keys())>0: # fail member
                self.fail_dict += design_group.fail_dict
                self.fail_member.append((design_group.prop_name, design_group.frame_size, list(design_group.fail_dict.keys())))
        
        # Rebar - column design
        for prop_name in self.column_props:
            size=self.fprop_size[prop_name]
            design_group=DesignGroup()
            for frame_name in self.frame_dict[prop_name]:
                total_length = self.sapModel.Story.GetHeight(self.fstory_dict[frame_name])[0]
                objs=etabsClass.sapModel.DesignConcrete.GetSummaryResultsColumn(frame_name)
                column_design = ColumnDesign(objs, design_config, size, prop_name, total_length)
                design_group.append(column_design)
            design_group.calculate_rebar_weight()
            self.column_design_groups.append(design_group)
        self.rebar_weight += sum([design_group.rebar_weight for design_group in self.column_design_groups])
        self.design_failure += sum([design_group.fail_num for design_group in self.column_design_groups])
        for design_group in self.column_design_groups:
            if len(design_group.fail_dict.keys())>0: # fail member
                self.fail_dict += design_group.fail_dict
                self.fail_member.append((design_group.prop_name, design_group.frame_size, list(design_group.fail_dict.keys())))
        return None
    
    def try_design(self, design_sizes):
        for idx, prop_name in enumerate(self.fprop_size.keys()):
            self.fprop_size[prop_name]=(design_sizes[2*idx],design_sizes[2*idx+1])
        self.modify_frame()
        
        self.check_drift()
        output=[self.max_story_drift]
        
        self.check_design()
        output.append(self.concrete_weight)
        output.append(self.rebar_weight)
        
        output.append(sum([design_group.fail_num for design_group in self.beam_design_groups])+
                      sum([design_group.fail_num for design_group in self.column_design_groups]) )
        return output
    
    def close(self, bool_save = False):
        self.myETABSObject.ApplicationExit(bool_save)

### Design Groups

In [6]:
class BeamDesign:
    def __init__(self, DesignObj, config, frame_size, prop_name):
        self.name = DesignObj[1][0]
        self.frame_size = frame_size
        self.prop_name = prop_name
        self.long_dia = config.long_beam_dia
        self.conf_dia = config.conf_beam_dia
        self.conf_leg = config.beam_conf_leg
        self.concrete_cover = config.concrete_cover
        
        long_area = calc_area(self.long_dia)
        conf_area = calc_area(self.conf_dia) 

        d = frame_size[0] - (config.concrete_cover+config.conf_beam_dia+config.long_beam_dia+12.5)
        min_spacing = min(d/4, 300)
        min_spacing2 = min(d/2, 600) # if steel is not needed as much 
        
        self.total_length=max(DesignObj[2])+min(DesignObj[2])
        end_idxs=[idx for idx, length in enumerate(DesignObj[2]) 
                  if (length/self.total_length<=1/3 or length/self.total_length>=2/3)]
        mid_idxs=[idx for idx in range(0,len(DesignObj[2])) if idx not in end_idxs]
        end_top=[]
        end_bot=[]
        end_shear=[]
        for idx in end_idxs:
            torsion_l=DesignObj[10][idx]
            end_top.append(DesignObj[4][idx]+torsion_l/2) #
            end_bot.append(DesignObj[6][idx]+torsion_l/2) #
            torsion_t=DesignObj[12][idx]
            end_shear.append(DesignObj[8][idx]+self.conf_leg*torsion_t) #mm2/mm
            
        mid_top=[]
        mid_bot=[]
        mid_shear=[]   
        for idx in mid_idxs:
            torsion_l=DesignObj[10][idx]
            mid_top.append(DesignObj[4][idx]+torsion_l/2) #
            mid_bot.append(DesignObj[6][idx]+torsion_l/2) #
            torsion_t=DesignObj[12][idx]
            mid_shear.append(DesignObj[8][idx]+self.conf_leg*torsion_t) #mm2/mm

        long_end_top=max(end_top)
        long_end_bot=max(end_bot)
        link_end=max(end_shear)
        long_mid_top=max(mid_top)
        long_mid_bot=max(mid_bot)
        link_mid=max(mid_shear)

        self.design_error_num =  sum([msg!='' for msg in DesignObj[-3]])
        self.design_error_dict= Counter([msg for msg in DesignObj[-3] if msg!=''])
        
        avail_length = frame_size[1] - 2*self.concrete_cover - 2*self.conf_dia - self.long_dia
        max_n_long = 2 * math.floor(avail_length /(40 + self.long_dia)) # max 1 stack
        
        self.n_end_top=max(math.ceil(long_end_top/(long_area)),2+self.conf_leg-2)
        self.n_end_bot=max(math.ceil(long_end_bot/(long_area)),2+self.conf_leg-2)
        self.n_mid_top=max(math.ceil(long_mid_top/(long_area)),2+self.conf_leg-2)
        self.n_mid_bot=max(math.ceil(long_mid_bot/(long_area)),2+self.conf_leg-2)
        
        if max(self.n_end_top,self.n_end_bot, self.n_mid_top, self.n_mid_bot)>max_n_long:
            self.long_space_req = 0
        else:
            self.long_space_req = 1
        
        if link_end != 0:
            self.s_end= floor_to_multiple(self.conf_leg*conf_area/link_end, 50) #min(self.conf_leg*conf_area/link_end, min_spacing)
        else:
            self.s_end = floor_to_multiple(min_spacing2, 50)

        if link_mid != 0:
            self.s_mid= floor_to_multiple(self.conf_leg*conf_area/link_mid, 50) #min(self.conf_leg*conf_area/link_mid, min_spacing2)
        else:
            self.s_mid = floor_to_multiple(min_spacing2, 50)
        

class ColumnDesign:
    def __init__(self, DesignObj, config, frame_size, prop_name, total_length):            
        self.name = DesignObj[1][0]
        self.frame_size = frame_size
        self.prop_name = prop_name
        self.design_error_num =  sum([msg!='' for msg in DesignObj[-3]])
        self.design_error_dict= Counter([msg for msg in DesignObj[-3] if msg!=''])
        
        self.long_dia = config.long_col_dia
        self.conf_dia = config.conf_col_dia
        self.conf_leg = config.col_conf_leg
        self.concrete_cover = config.concrete_cover
        
        long_area = calc_area(self.long_dia)
        conf_area = calc_area(self.conf_dia) 
        
        clear_length = max(DesignObj[3])
        self.joint_length = total_length-clear_length
        self.total_length = total_length

        # longitudinal bar
        long_steel=max(DesignObj[5])
        n_long=max(math.ceil(long_steel/long_area),4+(self.conf_leg-2)*4)
        # should check spacing > 40 mm

        # link bar
        self.l0=max(max(frame_size),1/6*clear_length,450) # support region
        n_side=math.ceil((n_long+4)/4)
        self.n_long=n_side*4-4
        
        self.long_spacing = (min(frame_size)-2*self.concrete_cover-2*self.conf_dia-self.long_dia)/(n_side-1)
        
        hx=(max(frame_size)-2*self.concrete_cover-self.conf_dia)/(self.conf_leg-1)
        s0=min(max(100+(350 - hx)/3,100),150)
        min_spacing_mid=min(6*self.long_dia,150)
        min_spacing_end=min(6*self.long_dia, 0.25*min(frame_size), s0)

        link_steel_mid=DesignObj[8][1]
        link_steel_end=max(DesignObj[8][0],DesignObj[8][-1])

        if link_steel_mid !=0:
            self.s_mid = floor_to_multiple(self.conf_leg*conf_area/link_steel_mid, 50) # min(self.conf_leg*conf_area/link_steel_mid,min_spacing_mid)
        else:
            self.s_mid = floor_to_multiple(min_spacing_mid,50)
        if link_steel_end !=0:
            self.s_end = floor_to_multiple(self.conf_leg*conf_area/link_steel_end, 50) # min(self.conf_leg*conf_area/link_steel_end,min_spacing_end)
        else:
            self.s_end = floor_to_multiple(min_spacing_end,50)
        
        
class DesignGroup: 
    def __init__(self):
        self.designs = []
        # obtained from members
        self.long_dia = None
        self.conf_dia = None
        self.conf_leg = None
        
        self.rebar_weight = 0
        self.fail_num = 0
        self.fail_dict = Counter([])
        
        # beam
        self.n_end_top = None
        self.n_end_bot = None
        self.n_mid_top = None
        self.n_mid_bot = None
        
        # column
        self.n_long = None
        self.l0= None
        
        self.s_end = None
        self.s_mid = None
    
    def __len__(self):
        return len(self.designs)
    
    def __iter__(self):
        return self.designs.__iter__()
    
    def extend(self, new_design):
        self.designs.extend(new_design)
    
    def append(self, new_design):
        self.designs.append(new_design)
        
    def find_design(self):
        self.long_dia = self.designs[0].long_dia
        self.conf_dia = self.designs[0].conf_dia
        self.conf_leg = self.designs[0].conf_leg
        self.prop_name = self.designs[0].prop_name
        self.frame_size = self.designs[0].frame_size
        
        if hasattr(self.designs[0], 'n_end_top'): # beam
            self.n_end_top=max([design.n_end_top for design in self.designs])
            self.n_end_bot=max([design.n_end_bot for design in self.designs])
            self.n_mid_top=max([design.n_mid_top for design in self.designs])
            self.n_mid_bot=max([design.n_mid_bot for design in self.designs])
            self.long_space_req = min([design.long_space_req for design in self.designs])
            if self.long_space_req == 0:
                self.fail_num += 1
                self.fail_dict['Not enough longitudinal spacing']=self.fail_dict['Not enough longitudinal spacing']+1
            
        else: # column
            self.n_long=max([design.n_long for design in self.designs])
            self.long_spacing = min([design.long_spacing for design in self.designs])
            if self.long_spacing <= 40 + self.long_dia:
                self.fail_num += 1
                self.fail_dict['Not enough longitudinal spacing']=self.fail_dict['Not enough longitudinal spacing']+1
        self.s_end=min([design.s_end for design in self.designs])
        self.s_mid=min([design.s_mid for design in self.designs])
    
    def calculate_rebar_weight(self):
        if self.n_end_top == None and self.n_long == None: # if both is None find design
            self.find_design()
        
        if self.s_end == 0:
            self.s_end = 1e-3
            self.fail_num += 1
            self.fail_dict['Not enough transversal reinforcement']=self.fail_dict['Not enough transversal reinforcement']+1
        
        if self.s_mid == 0:
            self.s_mid = 1e-3
            self.fail_num += 1
            self.fail_dict['Not enough transversal reinforcement']=self.fail_dict['Not enough transversal reinforcement']+1
        
        for design in self.designs:
            h=self.frame_size[0]-2*design.concrete_cover
            b=self.frame_size[1]-2*design.concrete_cover
            link_length=2*(b+h)+12*self.long_dia
            if self.n_end_top != None: # beam
                long_end_volume=(self.n_end_bot+self.n_end_top)*calc_area(self.long_dia)*2/3*design.total_length
                long_mid_volume=(self.n_mid_bot+self.n_mid_top)*calc_area(self.long_dia)*1/3*design.total_length
                long_volume=long_end_volume+long_mid_volume
                link_length+=(self.conf_leg-2)*(h+6*self.long_dia) 
                n_link_end=2*(math.ceil(design.total_length/3/self.s_end)+1) # n+1 points in n equal space
            else: # column
                long_volume=self.n_long*calc_area(self.long_dia)*design.total_length
                link_length+=(self.conf_leg-2)*(b+h+12*self.long_dia) 
                n_link_end_joint=(math.ceil((design.l0+design.joint_length)/self.s_end)+1) # n+1 points in n equal space
                n_link_end=(math.ceil(design.l0/self.s_end)+n_link_end_joint+1) # n+1 points in n equal space
                
            leftover_length=design.total_length-(n_link_end-2)*self.s_end # n space -1 for each edge
            n_link_mid=math.ceil(leftover_length/self.s_mid)-1 # don't count edges
            n_link=n_link_end+n_link_mid

            link_volume = calc_area(self.conf_dia)*link_length*n_link 

            rebar_volume = long_volume+link_volume
            self.rebar_weight += rebar_volume*steel_density/1e9 # kg/mm3
            
            self.fail_num += design.design_error_num
            self.fail_dict += design.design_error_dict
    
    def print_design_all(self):
        text1="D"+str(self.long_dia)
        text2="D"+str(self.conf_dia)+"-"
        if self.n_end_top != None: # beam
            for design in self.designs:
                print("Frame "+design.name)
                print("End top bar         ", str(design.n_end_top)+text1)
                print("End bottom bar      ", str(design.n_end_bot)+text1)
                print("Middle top bar      ", str(design.n_mid_top)+text1)
                print("Middle bottom bar   ", str(design.n_mid_bot)+text1)
                print("End link spacing    ",text2+str(design.s_end))
                print("Middle link spacing ",text2+str(design.s_mid))
                print("--------------------------------")
        else:
            for design in self.designs:
                print("Frame "+design.name)
                print("Longitudinal bar    ", str(design.n_long)+text1)
                print("End link spacing    ",text2+str(design.s_end))
                print("Middle link spacing ",text2+str(design.s_mid))
                print("--------------------------------")
            
def print_design_groups(design_groups):
    for design_group in design_groups:
        text1="D"+str(design_group.long_dia)
        text2="D"+str(design_group.conf_dia)+"-"
        print(design_group.prop_name)
        if design_group.n_end_top != None: # beam
            print("End top bar         ", str(design_group.n_end_top)+text1)
            print("End bottom bar      ", str(design_group.n_end_bot)+text1)
            print("Middle top bar      ", str(design_group.n_mid_top)+text1)
            print("Middle bottom bar   ", str(design_group.n_mid_bot)+text1)
        else:
            print("Longitudinal bar    ", str(design_group.n_long)+text1)
        print("End link spacing    ",text2+str(design_group.s_end))
        print("Middle link spacing ",text2+str(design_group.s_mid))
        print("Total Rebar Weight  ",str(round(design_group.rebar_weight,2))," Kg")
        print("Failure count       ",str(design_group.fail_num))
        print("--------------------------------")

# Metaheuristic

In [7]:
class Archive:
    def __init__(self, population, fitness, max_solution):
        self.population = population
        self.fitness = fitness.reshape(-1)
        self.max_solution = max_solution
        self.worst_individual_fitness = None
        self.worst_individual_idx = None  
        
        idx=np.argsort(self.fitness)[:max_solution]
        self.population=population[idx,:]
        self.fitness=fitness[idx]
        self.worst_individual_fitness=self.fitness[-1]
        self.worst_individual_idx=max_solution-1
    
    def __len__(self):
        return len(self.population)
    
    def update_archive(self, new_individual, new_fitness):
        if not any([all(arr == new_individual)  for arr in self.population]):
            if len(self.fitness) == self.max_solution:
                if new_fitness<self.worst_individual_fitness:
                    self.population[self.worst_individual_idx]=new_individual
                    self.fitness[self.worst_individual_idx]=new_fitness

                    idx=np.argmax(self.fitness)
                    self.worst_individual_idx=idx
                    self.worst_individual_fitness=self.fitness[idx]
            else:
                self.population=np.append(self.population,new_individual.reshape(1,-1),axis=0)
                self.fitness=np.append(self.fitness,new_fitness)

                if new_fitness>self.worst_individual_fitness:
                    self.worst_individual_idx=len(self.fitness)-1
                    self.worst_individual_fitness=new_fitness
                    
    def pick_k_means(self, n_total, n_best):
        n_cluster=n_total-n_best # remove number of cluster with best solution
        
        # remove n best from population
        partition_idx= np.argpartition(self.fitness,n_best)
        remain_idx = partition_idx[n_best:]  # take right side of partition (larger) 
        remaining_pop = self.population[remain_idx]
        
        # clustered pop
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(remaining_pop)
        kmeans = KMeans(
            init="random",
            n_clusters=n_cluster, 
            n_init=10,
            max_iter=300,
        )
        km=kmeans.fit(scaled_features)
        indices=[np.argmin(km.transform(scaled_features)[:,i]) for i in range(n_cluster)]
        clustered_pop=remaining_pop[indices]
        
        if n_best>0:
            # best pop 
            best_idx = partition_idx[:n_best]  # take left side of partition (smaller) 
            best_pop = self.population[best_idx]
            population = np.append(clustered_pop, best_pop, axis=0)
        else:
            population = clustered_pop     
        return population

## replace out of boundary solution stick to wall
def rangeCheck(x,ub,lb):
    for i in range(0,x.size):
        if x[i]<lb[i]:
            x[i]=lb[i]
        if x[i]>ub[i]:
            x[i]=ub[i]

## replace out of boundary solution continuous world 
def rangeCheck2(x,ub,lb):
    for i in range(0,x.size):
        while x[i]<lb[i]:
            x[i]=ub[i]-(lb[i]-x[i])
        while x[i]>ub[i]:
            x[i]=lb[i]+(x[i]-ub[i])

# SOS

In [8]:
# normal sos
def SOS(popsize,maxiter,ub,lb,objFun,verbose=True,n_archive=None):
    if n_archive == None:
        n_archive=popsize
    nvar=ub.shape[0]
    x=np.zeros([popsize,nvar])
    fit=np.zeros([popsize,1])

    ## ecosystem initialization
    for i in range(0,popsize):
        if i<0:#popsize/2:
            rand=np.random.triangular(0,1,1,size=(1,nvar))
        else:
            rand=np.random.rand(1,nvar)
        x[i,:]=rand*(ub-lb)+lb
        fit[i,:]=objFun(x[i,:])
    
    idx=np.argmin(fit)
    bestfit=fit[idx,:]
    bestx=x[idx,:]
    print(bestfit)
    
    archive=Archive(x,fit,n_archive)
    ## main loop
    bestfit=fit[idx,:]
    bestx=x[idx,:]
    recordfit=np.zeros(maxiter+1)
    recordfit[0]=bestfit
    for iteration in range(0,maxiter):
        for i in range(0,popsize):
            ## update best organism
            idx=np.argmin(fit)
            bestfit=fit[idx,:]
            bestx=x[idx,:]

            ## mutualism phase
            # choose organism j
            j=r.randrange(0,popsize)
            while j==i:
                j=r.randrange(0,popsize)
            # determine mutual vector and beneficial factor
            mutvec=(x[i,:]+x[j,:])/2
            BF1=r.randrange(1,3) # random integer between 1 and 2
            BF2=r.randrange(1,3)
            rand1=np.random.rand(1,nvar)
            rand2=np.random.rand(1,nvar)
            # calculate new solution
            xnew1=x[i,:]+rand1[0]*(bestx-BF1*mutvec)
            xnew2=x[j,:]+rand2[0]*(bestx-BF2*mutvec)
            # replace out of boundary solution
            rangeCheck(xnew1,ub,lb)
            rangeCheck(xnew2,ub,lb)
            # evaluate the fitness
            fitnew1=objFun(xnew1)
            fitnew2=objFun(xnew2)
            # accept new solution if the fitness is better
            if fitnew1<fit[i,0]:
                fit[i,0]=fitnew1
                x[i,:]=xnew1
                archive.update_archive(xnew1,fitnew1)
            if fitnew2<fit[j,0]:
                fit[j,0]=fitnew2
                x[j,:]=xnew2
                archive.update_archive(xnew2,fitnew2)
            ## commensalism phase
            # choose organism j
            j=r.randrange(0,popsize)
            while j==i:
                j=r.randrange(0,popsize)
            rand=np.random.rand(1,nvar)*2-1 # random between 0 and 1
            # calculate new solution
            xnew1=x[i,:]+rand[0]*(bestx-x[j,:])
            # replace out of boundary solution
            rangeCheck(xnew1,ub,lb)
            # evaluate the fitness
            fitnew1=objFun(xnew1)
            #accept new solution if the fitness is better
            if fitnew1<fit[i,0]:
                fit[i,0]=fitnew1
                x[i,:]=xnew1
                archive.update_archive(xnew1,fitnew1)

            ## parasitism phase
            # choose organism j
            j=r.randrange(0,popsize)
            while j==i:
                j=r.randrange(0,popsize)
            parasite=copy.copy(x[i,:]) # copy organism i
            # select variable to mutate
            permut=np.random.permutation(nvar)
            idx=permut[0:r.randrange(1,nvar+1)]
            rand=np.random.rand(1,idx.shape[0])
            parasite[idx]=rand*(ub[idx]-lb[idx])+lb[idx]
            rangeCheck(parasite,ub,lb)
            # evaluate the fitness
            fitpar=objFun(parasite) 
            #accept new solution if the fitness is better
            if fitpar<fit[j,0]:
                fit[j,0]=fitpar
                x[j,:]=parasite
                archive.update_archive(parasite,fitpar)
        recordfit[iteration+1]=copy.copy(bestfit)
        if verbose:
            print("Iter {}: {}".format(iteration,bestfit))
    ## update best organism
    idx=np.argmin(fit)
    bestfit=fit[idx,:]
    bestx=x[idx,:]
    return bestx, recordfit, archive

In [10]:
# sos+screening+update parasitism
def DGBM(popsize,maxiter,ub,lb,objFun,n_round, boolParasite = False):
    nvar=ub.shape[0]
    x=np.zeros([popsize,nvar])
    fit=np.zeros([popsize,1])
    fail_member=[[] for _ in range(popsize)]
    
    func_call=0
    ## ecosystem initialization
    for i in range(0,popsize):
        if i<0:#popsize/2:
            rand=np.random.triangular(0,1,1,size=(1,nvar))
        else:
            rand=np.random.rand(1,nvar)
        x[i,:]=rand*(ub-lb)+lb
        fit[i,:],fail_member[i] = objFun(x[i,:])
        func_call = func_call + 1
     
    ## main loop
    recordfit=np.zeros(maxiter)
    recordfunc=np.zeros(maxiter)
    
    iteration = 0
    func_call_train = 0
    while iteration != maxiter:
        iteration_scaler = (iteration-n_round)/(maxiter-n_round)
        if iteration == n_round:
            LGB_models = train_LGB(DoE)
            func_call_train = func_call
        for i in range(0,popsize):
            ## update best organism
            idx=np.argmin(fit)
            bestfit=fit[idx,:]
            bestx=x[idx,:]

            ## mutualism phase
            # choose organism j
            j=r.randrange(0,popsize)
            while j==i:
                j=r.randrange(0,popsize)
            # determine mutual vector and beneficial factor
            mutvec=(x[i,:]+x[j,:])/2
            BF1=r.randrange(1,3) # random integer between 1 and 2
            BF2=r.randrange(1,3)
            rand1=np.random.rand(1,nvar)
            rand2=np.random.rand(1,nvar)
            # calculate new solution
            xnew1=x[i,:]+rand1[0]*(bestx-BF1*mutvec)
            xnew2=x[j,:]+rand2[0]*(bestx-BF2*mutvec)
            # replace out of boundary solution
            rangeCheck(xnew1,ub,lb)
            rangeCheck(xnew2,ub,lb)
            
            # evaluate normally if within n_round
            boolCheck1 = False
            boolCheck2 = False
            if iteration < n_round:
                boolCheck1 = True
                boolCheck2 = True
            else:
                if check_candidate(LGB_models, xnew1, iteration_scaler, bestfit):
                    boolCheck1 = True
                if check_candidate(LGB_models, xnew2, iteration_scaler, bestfit):
                    boolCheck2 = True
                
            if boolCheck1:
                # evaluate the fitness
                fitnew1,fail_member_new1=objFun(xnew1)
                # accept new solution if the fitness is better
                if fitnew1<fit[i,0]:
                    fit[i,0]=fitnew1
                    fail_member[i]=fail_member_new1
                    x[i,:]=xnew1
                func_call = func_call + 1
            if boolCheck2:
                fitnew2,fail_member_new2=objFun(xnew2)
                if fitnew2<fit[j,0]:
                    fit[j,0]=fitnew2
                    fail_member[j]=fail_member_new2
                    x[j,:]=xnew2
                func_call = func_call + 1
            
            if iteration >= n_round and func_call_train < func_call:
                LGB_models = train_LGB(DoE)
                func_call_train = func_call
        
            ## commensalism phase
            # choose organism j
            j=r.randrange(0,popsize)
            while j==i:
                j=r.randrange(0,popsize)
            rand=np.random.rand(1,nvar)*2-1 # random between 0 and 1
            # calculate new solution
            xnew1=x[i,:]+rand[0]*(bestx-x[j,:])
            # replace out of boundary solution
            rangeCheck(xnew1,ub,lb)
            
            boolCheck1 = False
            # evaluate normally if within n_round
            if iteration < n_round:
                boolCheck1 = True
            elif check_candidate(LGB_models, xnew1, iteration_scaler, bestfit):
                boolCheck1 = True
                
            if boolCheck1:
                # evaluate the fitness
                fitnew1,fail_member_new1=objFun(xnew1)
                #accept new solution if the fitness is better
                if fitnew1<fit[i,0]:
                    fit[i,0]=fitnew1
                    fail_member[i]=fail_member_new1
                    x[i,:]=xnew1
                func_call = func_call + 1
            
            if iteration >= n_round and func_call_train < func_call:
                LGB_models = train_LGB(DoE)
                func_call_train = func_call

            ## parasitism phase
            # choose organism j
            j=r.randrange(0,popsize)
            while j==i:
                j=r.randrange(0,popsize)
            parasite=copy.copy(x[i,:]) # copy organism i
            
            if len(fail_member[i]) == 0 or not boolParasite:
                # select variable to mutate
                permut=np.random.permutation(nvar)
                idx=permut[0:r.randrange(1,nvar+1)]
                rand=np.random.rand(1,idx.shape[0])
                parasite[idx]=rand*(ub[idx]-lb[idx])+lb[idx]
            else:
                parasite=modify_fail(parasite,fail_member[i])
            # replace out of boundary solution
            rangeCheck(parasite,ub,lb)
            
            boolCheck1 = False
            # evaluate normally if within n_round
            if iteration < n_round:
                boolCheck1 = True
            elif check_candidate(LGB_models, parasite, iteration_scaler, bestfit):
                boolCheck1 = True
            
            if boolCheck1 == True:
                # evaluate the fitness
                fitpar,fail_member_par=objFun(parasite)
                #accept new solution if the fitness is better
                if fitpar<fit[j,0]:
                    fit[j,0]=fitpar
                    fail_member[j]=fail_member_par
                    x[j,:]=parasite
                func_call = func_call + 1
            
            if iteration >= n_round and func_call_train < func_call:
                LGB_models = train_LGB(DoE)
                func_call_train = func_call
                
        if func_call >= popsize+4*popsize*iteration:
            recordfit[iteration]=copy.copy(bestfit)
            recordfunc[iteration]=copy.copy(func_call)
            print("Iter {}-{}: {}".format(iteration,func_call,bestfit))
            iteration+=1
    
    ## update best organism
    idx=np.argmin(fit)
    bestfit=fit[idx,:]
    bestx=x[idx,:]
    ## print out result
    print(bestfit)
    print(bestx)
    return bestx, recordfit, recordfunc

In [15]:
# concrete euro 100/2400 (0.416) /kg, steel 1.2 euro/kg
# https://www.sciencedirect.com/science/article/pii/S0141029616308768
def modify_input(dsize):
    # modift input to ensure stacking column size and deep beam
    # column
    sizes=[]
    for idx in columns_ext_idx:
        depth=dsize[2*idx]
        width=dsize[2*idx+1]
        new_size=round_to_multiple((depth+width)/2,50)
        sizes.extend([new_size,new_size])
    sizes.sort(reverse=True)
    for i, idx in enumerate(columns_ext_idx):
        dsize[2*idx]=sizes[2*i]
        dsize[2*idx+1]=sizes[2*i]

    sizes=[]
    for idx in columns_int_idx:
        depth=dsize[2*idx]
        width=dsize[2*idx+1]
        new_size=round_to_multiple((depth+width)/2,50)
        sizes.extend([new_size,new_size])
    sizes.sort(reverse=True)
    for i, idx in enumerate(columns_int_idx):
        dsize[2*idx]=sizes[2*i]
        dsize[2*idx+1]=sizes[2*i]

    # beam
    # ACI 318-19 18.6.2.1
    # width/height < 0.3
    for idx in beams_idx:
        depth=round_to_multiple(dsize[2*idx],50)
        width=round_to_multiple(dsize[2*idx+1],50)
        ######### CHANGE HERE
        story_num=int(names[idx][1])-1
        story_col_size=sizes[2*story_num] # compare with size of interior column
        if depth<width:
            dsize[2*idx]=width
            dsize[2*idx+1]=min(depth, story_col_size)
        else:
            dsize[2*idx]=depth
            dsize[2*idx+1]=min(width, story_col_size)
        # bw ratio >= 0.3
        min_width=ceil_to_multiple(0.3*dsize[2*idx],50)
        if dsize[2*idx+1]<min_width:
            dsize[2*idx+1]=min(min_width, story_col_size)
    return dsize

In [16]:
def modify_fail(dsize, fail_idx):
    # modify dimension of failed section
    for idx in fail_idx:
        depth=dsize[2*idx]
        width=dsize[2*idx+1]
        dsize[2*idx]=depth+round_to_multiple(r.triangular(0,1,0)*
                                          (upper_bound[2*idx]-depth),50)
        dsize[2*idx+1]=width+round_to_multiple(r.triangular(0,1,0)*
                                          (upper_bound[2*idx+1]-width),50)
    return modify_input(dsize)

# check candidate return true if it is worth checking
def check_candidate(LGB_models, x, iteration_scaler, bestcost):
    # retrain using DoE
    mod_dsize=modify_input(x)
    
    safe_thres = safe_proba_threshold[0] + (safe_proba_threshold[1]-safe_proba_threshold[0])*iteration_scaler
    thres_mult = threshold_multiplier[0] + (threshold_multiplier[1]-threshold_multiplier[0])*iteration_scaler
    if len(LGB_models) <= 4:
        # predict model in outputs
        output=predict_LGB2(LGB_models,mod_dsize.reshape(1,-1))
        drift=output[0]
        concrete_weight=output[1]
        steel_weight=output[2]
        safe_state_prob=output[-1]

        if safe_state_prob > safe_thres and drift < allowable_drift * thres_mult:
            cost=conc_cost*concrete_weight + steel_cost*steel_weight
            if cost < thres_mult * bestcost:
                return True
    else:
        output = predict_LGB_plus(LGB_models,mod_dsize.reshape(1,-1))
        drift=output[0]
        concrete_weight=output[1]
        steel_weight=output[2]
        safe_state_prob=output[3:]
        # all safe
        if min(safe_state_prob) > safe_thres and drift < allowable_drift * thres_mult:
            cost=conc_cost*concrete_weight + steel_cost*steel_weight
            if cost < thres_mult * bestcost:
                return True
    return False

In [17]:
# must iterate 1 row at a time
def objFunc(x):
    global DoE
    mod_dsize=modify_input(x)
    
    output=etabsClass.try_design(mod_dsize)
    drift=output[0]
    concrete_weight=output[1]
    steel_weight=output[2]
    fail_num=output[-1]    
    
    fail_names = [x[0] for x in etabsClass.fail_member]
    fail_idx= [idx for idx, name in enumerate(names) if name in fail_names]
    
    DoE=DoE+[(mod_dsize,output)]
    
    cost=conc_cost*concrete_weight+steel_cost*steel_weight
    penalty=0
    if drift>allowable_drift:
        penalty+=cost*(2+(drift-allowable_drift)*1000)
    if fail_num>0:
        penalty+=cost*(2+len(fail_idx)/len(names)*3)
    return cost+penalty

# give obj func + fail member idx
def objFunc_plus(x):
    global DoE
    mod_dsize=modify_input(x)
    output=etabsClass.try_design(mod_dsize)
    drift=output[0]
    concrete_weight=output[1]
    steel_weight=output[2]
    fail_num=output[-1]    
    
    fail_names = [x[0] for x in etabsClass.fail_member]
    fail_idx= [idx for idx, name in enumerate(names) if name in fail_names]
    
    DoE=DoE+[(mod_dsize,output)]
    
    cost=conc_cost*concrete_weight+steel_cost*steel_weight
    penalty=0
    if drift>allowable_drift:
        penalty+=cost*(2+(drift-allowable_drift)*1000)
    if fail_num>0:
        penalty+=cost*(2+len(fail_idx)/len(names)*3)
    return cost+penalty, fail_idx

# give obj func + fail member idx and include in DoE
def objFunc_plus_DoE(x):
    global DoE
    mod_dsize=modify_input(x)
    output=etabsClass.try_design(mod_dsize)
    drift=output[0]
    concrete_weight=output[1]
    steel_weight=output[2]
    fail_num=output[-1]    
    
    fail_names = [x[0] for x in etabsClass.fail_member]
    fail_idx= [idx for idx, name in enumerate(names) if name in fail_names]
    
    # make DoE with output+fail_idx
    output[-1]=[0 if (idx in fail_idx) else 1 for idx, name in enumerate(names)] # 0 fail, 1 safe
    DoE=DoE+[(mod_dsize,output)]
    
    cost=conc_cost*concrete_weight+steel_cost*steel_weight
    penalty=0
    if drift>allowable_drift:
        penalty+=cost*(2+(drift-allowable_drift)*1000)
    if fail_num>0:
        penalty+=cost*(2+len(fail_idx)/len(names)*3)
    return cost+penalty, fail_idx

# use LGB to calculate objective function
def objFunc2(LGB_models,x):
    mod_dsize=modify_input(x)
    # predict model in outputs
    output=predict_LGB(LGB_models,mod_dsize.reshape(1,-1))
    drift=output[0]
    concrete_weight=output[1]
    steel_weight=output[2]
    safe_state=output[-1]
    
    cost=conc_cost*concrete_weight+steel_cost*steel_weight
    penalty=0
    if drift>allowable_drift:
        penalty+=cost*(2+(drift-allowable_drift)*1000)
    if safe_state==0:
        penalty+=cost*2
    return cost+penalty

In [18]:
# combined safety output
def train_LGB1(DoE):
    design_sizes=[point[0] for point in DoE]
    outputs=np.array([point[1] for point in DoE])
    binary_out=outputs[:,-1]==0 # safe==1
    
    LGB_models=[]
    for col, output in enumerate(outputs.T[:-1]):
        model=lgb.LGBMRegressor(verbose = -1)
        model.fit(design_sizes,output)
        LGB_models.append(model)

    # classification
    model=lgb.LGBMClassifier(verbose = -1)
    model.fit(design_sizes,binary_out)
    LGB_models.append(model)
    return LGB_models

# divided safety output per group
def train_LGB2(DoE):
    design_sizes=[point[0] for point in DoE]
    pred_out=np.array([point[1][:3] for point in DoE])
    bin_out =np.array([point[1][-1] for point in DoE])
    
    LGB_models=[]
    # 0 drift, 1 concrete weight, 2 steel weight 
    for col, output in enumerate(pred_out.T): 
        model=lgb.LGBMRegressor(verbose = -1)
        model.fit(design_sizes, output)
        LGB_models.append(model)
    
    # classification
    for col, output in enumerate(bin_out.T):
        model=lgb.LGBMClassifier(verbose = -1)
        model.fit(design_sizes, output)
        LGB_models.append(model)
    return LGB_models

# wrapper function for train LGB
def train_LGB(DoE):
    # if the DoE has divided the classification of safety
    if not hasattr(DoE[0][1][-1], "__len__"):
        return train_LGB1(DoE)
    else:
        return train_LGB2(DoE)

# define LGB_models
def predict_LGB(LGB_models,x):
    x=x.reshape(1,-1)
    pred_out=np.zeros(len(LGB_models))
    for col, model in enumerate(LGB_models):
        pred_out[col] = model.predict(x)
    return pred_out

# give prediction probability for clasification
def predict_LGB2(LGB_models,x):
    x=x.reshape(1,-1)
    pred_out=np.zeros(len(LGB_models))
    for col, model in enumerate(LGB_models[:-1]):
        pred_out[col] = model.predict(x)
    proba = LGB_models[-1].predict_proba(x)
    pred_out[col+1] = proba[0][1] # probability of classifying 1/safe
    return pred_out

# give prediction probability for clasification
def predict_LGB_plus(LGB_models,x):
    x=x.reshape(1,-1)
    pred_out=np.zeros(len(LGB_models))
    # prediction
    for col, model in enumerate(LGB_models[:3]):
        pred_out[col] = model.predict(x)
    # predict_proba
    for col, model in enumerate(LGB_models[3:]):
        proba = LGB_models[3+col].predict_proba(x)
        pred_out[3+col] = proba[0][1] # probability of classifying 1/safe
    return pred_out

# Framework

In [21]:
etabsClass=ETABSObjectiveFunction()

In [22]:
upper_bound=[]
lower_bound=[]
for idx, prop_name in enumerate(etabsClass.fprop_size.keys()):
    if prop_name[0]=='B':
        upper_bound.extend([1000,1000])
        lower_bound.extend([300,300])
    else:
        upper_bound.extend([1000,1000])
        lower_bound.extend([300,300])
upper_bound=np.array(upper_bound)
lower_bound=np.array(lower_bound)

ndim=len(upper_bound)

# for modifying input
names=list(etabsClass.fprop_size.keys())
columns_ext_idx=[idx for idx, name in enumerate(names) if name[0]=='C' and name[-2:-1]=='x']
columns_int_idx=[idx for idx, name in enumerate(names) if name[0]=='C' and name[-2:-1]=='n']
beams_idx=[idx for idx, name in enumerate(names) if name[0]=='B']

In [23]:
DoE=[]
bestx, record_fit, record_func = DGBM(20, 50, upper_bound, lower_bound, objFunc_plus, 10, True)